⚠️ Note: This project is currently in active development and is subject to significant modifications. We could also use RAG, and that could be better obviously. Here, I am just testing fine-tuning and different methods.



In [ ]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model

import torch

In [ ]:
!git clone https://github.com/huggingface/smolagents.git

fatal: destination path 'smolagents' already exists and is not an empty directory.


In [ ]:
def collect_files(directory, extensions=['.py', '.ipynb']):
    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.endswith(ext) for ext in extensions):
                file_paths.append(os.path.join(root, file))
    return file_paths

repo_directory = "smolagents"
examples = []
code_files = collect_files(repo_directory)

In [ ]:
for file in code_files:
    with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
        examples.append({"text": content})

In [ ]:
dataset = Dataset.from_list(examples)
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
base_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
if base_tokenizer.pad_token is None:
    base_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
def tokenize_function(example):
    tokenized = base_tokenizer(example['text'], truncation=True, max_length=1000, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=base_tokenizer, mlm=False)
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)



# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.resize_token_embeddings(len(base_tokenizer))

Embedding(32001, 4096)

In [ ]:
lora_config = LoraConfig(
    r=8,                      # LoRA rank
    lora_alpha=32,            # Scaling factor for LoRA
    target_modules=["q_proj", "v_proj"],  # Adjust targets for LLaMA architecture
    lora_dropout=0.1,
    bias="none"
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./fine-tuned-llama",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,                       # Enable mixed precision training if supported
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    peft_config=lora_config
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TypeError: Trainer.__init__() got an unexpected keyword argument 'peft_config'

In [ ]:
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)


Evaluation Results: {'eval_runtime': 0.6315, 'eval_samples_per_second': 11.085, 'eval_steps_per_second': 3.167, 'epoch': 3.0}


In [ ]:
from transformers import pipeline

In [ ]:
prompt = "Write a sample code for smolagents library"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=base_tokenizer, max_length=1000)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

Device set to use cuda:0
The model 'PeftModel' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCa


```python
import smolagents

from smolagents.agents import SmolAgent

agent = SmolAgent(
    name="My Agent",
    description="This is a sample agent",
    train_on_inputs=True,
    train_on_actions=True,
    train_on_rewards=True,
    train_on_trajectories=True,
    train_on_trajectories_length=20,
    train_on_episodes=100,
    train_on_episodes_length=100,
    train_on_episodes_start_episode=1,
    train_on_episodes_end_episode=20,
    train_on_episodes_interval=1,
    train_on_episodes_interval_start_episode=1,
    train_on_episodes_interval_end_episode=20,
    train_on_episodes_interval_step=1,
    train_on_episodes_interval_start_episode_step=1,
    train_on_episodes_interval_end_episode_step=20,
    train_on_episodes_interval_step_max=100,
    train_on_episodes_interval_start_episode_step_max=1,
    train_on_episodes_interval_end_episode_step_max=20,
    train_on_episodes_interval_step_min=1,
    train_on_episodes_interval_start_episode_step_min=1,
    train_on_episodes_interva

In [ ]:
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

18520